In [1]:
## Installing WandB
!pip install wandb -qqq

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision.transforms import ToTensor

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

from tqdm import tqdm

Device:  cuda


In [1]:
import wandb, os
os.environ['WANDB_API_KEY'] = "5203e53880ceb7b6d2c0a93809e14ae43261f2ed" #your key here
wandb.login()

wandb: Currently logged in as: cs24m016 (cs24m016-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 14.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found

# Question 1

In [5]:
import torch
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(
        self,
        input_shape=(3, 224, 224),
        conv_filters=[32, 64, 128, 256, 512],
        filter_sizes=[3, 3, 3, 3, 3],
        activation_fn=nn.ReLU,
        dense_units=256,
        dense_activation_fn=nn.ReLU,
        dropout_rate=0.3,
        batch_norm=True,
        num_classes=10
    ):
        super(ConvNet, self).__init__()

        self.conv_blocks = nn.Sequential()
        in_channels = input_shape[0]
        h, w = input_shape[1], input_shape[2]

        # Add 5 Conv-BN-Activation-Pool blocks
        for i in range(5):
            out_channels = conv_filters[i]
            kernel_size = filter_sizes[i]
            padding = kernel_size // 2  # keep same spatial size before pooling

            self.conv_blocks.add_module(f"conv{i+1}", nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding))
            if batch_norm:
                self.conv_blocks.add_module(f"bn{i+1}", nn.BatchNorm2d(out_channels))
            self.conv_blocks.add_module(f"act{i+1}", activation_fn())
            self.conv_blocks.add_module(f"pool{i+1}", nn.MaxPool2d(2))
            if dropout_rate > 0:
                self.conv_blocks.add_module(f"dropout{i+1}", nn.Dropout2d(dropout_rate))

            in_channels = out_channels
            h, w = h // 2, w // 2  # due to MaxPool2d(2)

        # Compute the flattened size after conv blocks
        self.flattened_size = in_channels * h * w

        self.fc1 = nn.Linear(self.flattened_size, dense_units)
        self.fc1_act = dense_activation_fn()
        self.dropout = nn.Dropout(dropout_rate)

        self.output_layer = nn.Linear(dense_units, num_classes)

    def forward(self, x):
        x = self.conv_blocks(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.fc1_act(self.fc1(x)))
        return self.output_layer(x)


# Question 2

In [5]:
!pip install wandb


In [2]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, Subset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

def get_dataloaders(data_dir, batch_size=64, val_split=0.2, augment=True):
    # Transforms
    train_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor()
    ]) if augment else transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    test_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    full_dataset = ImageFolder(root=data_dir, transform=train_transforms)

    # Stratified split
    targets = np.array(full_dataset.targets)
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=val_split, random_state=42)
    train_idx, val_idx = next(splitter.split(np.zeros(len(targets)), targets))

    train_set = Subset(full_dataset, train_idx)
    val_set = Subset(ImageFolder(root=data_dir, transform=test_transforms), val_idx)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader, len(full_dataset.classes)


In [4]:
import torch.nn.functional as F
import wandb

def train(model, train_loader, val_loader, optimizer, criterion, device, epochs=10):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss, correct = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(dim=1) == labels).sum().item()

        train_accuracy = correct / len(train_loader.dataset)

        # Validation
        model.eval()
        val_correct, val_loss = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                val_correct += (outputs.argmax(dim=1) == labels).sum().item()

        val_accuracy = val_correct / len(val_loader.dataset)

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": total_loss / len(train_loader),
            "train_accuracy": train_accuracy,
            "val_loss": val_loss / len(val_loader),
            "val_accuracy": val_accuracy
        })

        print(f"Epoch {epoch+1} - Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")


In [6]:
from torchvision import models
from torch import optim
import torch.nn as nn
import wandb


def main():
    wandb.init(project="DL_A2")

    config = wandb.config

    activation_map = {
        "ReLU": nn.ReLU,
        "GELU": nn.GELU,
        "SiLU": nn.SiLU,
        "Mish": nn.Mish
    }

    model = ConvNet(
        input_shape=(3, 224, 224),
        conv_filters=config.conv_filters,
        filter_sizes=config.filter_sizes,
        activation_fn=activation_map[config.activation_fn],
        dense_units=config.dense_units,
        dense_activation_fn=activation_map[config.activation_fn],
        dropout_rate=config.dropout,
        batch_norm=config.batch_norm,
        num_classes=10
    )

    train_loader, val_loader, _ = get_dataloaders(
        data_dir="/kaggle/input/nature-12k/inaturalist_12K/train",
        batch_size=config.batch_size,
        augment=config.augment
    )

    optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    train(model, train_loader, val_loader, optimizer, criterion, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), epochs=config.epochs)



In [7]:
sweep_config = {
    "method": "random",
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "conv_filters": {
            "values": [[32, 32, 64, 64, 128], [32, 64, 128, 256, 512]]
        },
        "filter_sizes": {
            "values": [[3, 3, 3, 3, 3]]
        },
        "activation_fn": {
            "values": ["ReLU", "GELU", "SiLU", "Mish"]
        },
        "dropout": {
            "values": [0.2, 0.3]
        },
        "dense_units": {
            "values": [128, 256]
        },
        "batch_norm": {
            "values": [True, False]
        },
        "augment": {
            "values": [True, False]
        },
        "batch_size": {
            "values": [64, 128]
        },
        "lr": {
            "values": [0.01, 0.001]
        },
        "epochs": {
            "value": 10
        }
    }
}


In [8]:
sweep_id = wandb.sweep(sweep=sweep_config, project='DL_A2')
wandb.agent(sweep_id, function=main, count=5)


Create sweep with ID: a08wpdkg
Sweep URL: https://wandb.ai/cs24m016-indian-institute-of-technology-madras/DL_A2/sweeps/a08wpdkg


wandb: Agent Starting Run: dg9yyb4x with config:
wandb: 	activation_fn: ReLU
wandb: 	augment: False
wandb: 	batch_norm: False
wandb: 	batch_size: 128
wandb: 	conv_filters: [32, 32, 64, 64, 128]
wandb: 	dense_units: 256
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.001


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 1 - Train Acc: 0.0964, Val Acc: 0.1000


wandb: Ctrl + C detected. Stopping sweep.


epoch,▁
train_accuracy,▁
train_loss,▁
val_accuracy,▁
val_loss,▁
epoch,1
train_accuracy,0.09639
train_loss,2.30339
val_accuracy,0.1
val_loss,2.3031


In [ ]:
sweep_id = wandb.sweep(sweep=sweep_config, project='DL_A2')
wandb.agent(sweep_id, function=main, count=50)


Create sweep with ID: g273f63a
Sweep URL: https://wandb.ai/cs24m016-indian-institute-of-technology-madras/DL_A2/sweeps/g273f63a


wandb: Agent Starting Run: pbjxyqp1 with config:
wandb: 	activation_fn: SiLU
wandb: 	augment: True
wandb: 	batch_norm: True
wandb: 	batch_size: 128
wandb: 	conv_filters: [32, 32, 64, 64, 128]
wandb: 	dense_units: 256
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.001


Epoch 1 - Train Acc: 0.1315, Val Acc: 0.1225
Epoch 2 - Train Acc: 0.1708, Val Acc: 0.1595
Epoch 3 - Train Acc: 0.1825, Val Acc: 0.1755
Epoch 4 - Train Acc: 0.2028, Val Acc: 0.2000
Epoch 5 - Train Acc: 0.2175, Val Acc: 0.2260
Epoch 6 - Train Acc: 0.2235, Val Acc: 0.2400
Epoch 7 - Train Acc: 0.2360, Val Acc: 0.2505
Epoch 8 - Train Acc: 0.2503, Val Acc: 0.2575
Epoch 9 - Train Acc: 0.2503, Val Acc: 0.2740
Epoch 10 - Train Acc: 0.2617, Val Acc: 0.2685


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▃▄▅▆▆▇▇▇█
train_loss,█▇▆▅▄▄▃▂▂▁
val_accuracy,▁▃▃▅▆▆▇▇██
val_loss,█▇▇▆▅▄▃▃▂▁
epoch,10
train_accuracy,0.26166
train_loss,2.0665
val_accuracy,0.2685
val_loss,2.05089


wandb: Agent Starting Run: 23qtgfmb with config:
wandb: 	activation_fn: GELU
wandb: 	augment: False
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	conv_filters: [32, 32, 64, 64, 128]
wandb: 	dense_units: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.01


...

wandb: Agent Starting Run: 2c9v2nrv with config:
wandb: 	activation_fn: Mish
wandb: 	augment: False
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	conv_filters: [32, 32, 64, 64, 128]
wandb: 	dense_units: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.01


Epoch 1 - Train Acc: 0.0938, Val Acc: 0.1000
Epoch 2 - Train Acc: 0.0950, Val Acc: 0.1000
Epoch 3 - Train Acc: 0.0969, Val Acc: 0.1000
Epoch 4 - Train Acc: 0.0983, Val Acc: 0.1000
Epoch 5 - Train Acc: 0.0995, Val Acc: 0.1005
Epoch 6 - Train Acc: 0.0939, Val Acc: 0.1045
Epoch 7 - Train Acc: 0.1034, Val Acc: 0.1000
Epoch 8 - Train Acc: 0.1115, Val Acc: 0.1200
Epoch 9 - Train Acc: 0.1063, Val Acc: 0.1140
Epoch 10 - Train Acc: 0.1240, Val Acc: 0.1625


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▁▂▂▂▁▃▅▄█
train_loss,██████▇▇▆▁
val_accuracy,▁▁▁▁▁▂▁▃▃█
val_loss,███████▇▇▁
epoch,10
train_accuracy,0.12402
train_loss,2.29439
val_accuracy,0.1625
val_loss,2.28365


wandb: Agent Starting Run: k9wru0t2 with config:
wandb: 	activation_fn: Mish
wandb: 	augment: True
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	conv_filters: [32, 64, 128, 256, 512]
wandb: 	dense_units: 256
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.01


Epoch 1 - Train Acc: 0.1591, Val Acc: 0.2280
Epoch 2 - Train Acc: 0.1948, Val Acc: 0.2045
Epoch 3 - Train Acc: 0.1996, Val Acc: 0.2480
Epoch 4 - Train Acc: 0.2123, Val Acc: 0.2685
Epoch 5 - Train Acc: 0.2279, Val Acc: 0.2290


In [11]:
!pip install GPUtil

from GPUtil import showUtilization as gpu_usage
gpu_usage()                             


  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=5d8823929d073a62a1e81df246b2f20a6b8a0ef94f272470c621333a8198f11f
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% | 91% |
|  1 |  0% |  0% |


In [12]:
import torch
torch.cuda.empty_cache()


In [13]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           


Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 91% |
|  1 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 84% |  1% |
|  1 |  0% |  0% |


# question 3 

In [2]:

sweep_config = {
    "method": "random",
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "conv_filters": {
            "values": [[32, 32, 64, 64, 128],[512,256,128,64,32],[256,128,64,64,32], [32, 64, 128, 256, 512]]
        },
        "filter_sizes": {
            "values": [[3, 3, 3, 3, 3],[5,5,5,5,5],[7,7,7,7,7],[7,7,5,5,3],[7,5,3,3,3]]
        },
        "activation_fn": {
            "values": ["ReLU", "GELU", "SiLU", "Mish"]
        },
        "dropout": {
            "values": [0.0,0.2, 0.3]
        },
        "dense_units": {
            "values": [128, 256]
        },
        "batch_norm": {
            "values": [True]
        },
        "augment": {
            "values": [True, False]
        },
        "batch_size": {
            "values": [64, 128,256]
        },
        "lr": {
            "values": [0.01, 0.001]
        },
        "epochs": {
            "value": 10
        }
    }
}



In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import wandb


def get_dataloaders(data_dir, batch_size=256, val_split=0.2, augment=True):
    # Enhanced data augmentation
    train_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]) if augment else transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transforms)
    
    # Stratified split
    targets = np.array(full_dataset.targets)
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=val_split, random_state=42)
    train_idx, val_idx = next(splitter.split(np.zeros(len(targets)), targets))
    
    train_set = Subset(full_dataset, train_idx)
    val_set = Subset(datasets.ImageFolder(root=data_dir, transform=val_transforms), val_idx)
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, 
                             num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, 
                           num_workers=4, pin_memory=True)
    
    return train_loader, val_loader, full_dataset.classes

class OptimizedCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(OptimizedCNN, self).__init__()
        
        # Larger filters in early layers, smaller in later layers
        self.conv_blocks = nn.Sequential(
            # Block 1: 64 filters, 7x7 kernel
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            
            # Block 2: 128 filters, 5x5 kernel
            nn.Conv2d(64, 128, kernel_size=5, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            
            # Block 3: 256 filters, 3x3 kernel
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            
            # Block 4: 512 filters, 3x3 kernel
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            
            # Block 5: 512 filters, 3x3 kernel
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # Classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, num_classes))
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, 0, 0.01)
                init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.conv_blocks(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

def train(model, train_loader, val_loader, optimizer, criterion, scheduler, device, epochs=20):
    model.to(device)
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        model.train()
        train_loss, train_correct = 0.0, 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_correct += predicted.eq(labels).sum().item()
        
        train_acc = 100 * train_correct / len(train_loader.dataset)
        
        # Validation
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                val_correct += outputs.argmax(1).eq(labels).sum().item()
        
        val_acc = 100 * val_correct / len(val_loader.dataset)
        
        # Step the scheduler
        scheduler.step(val_loss)
        
        # Log metrics
        wandb.log({            
            "epoch": epoch + 1,
            "train_loss": train_loss / len(train_loader),
            "train_accuracy": train_acc,
            "val_loss": val_loss / len(val_loader),
            "val_accuracy": val_acc,
            "lr": optimizer.param_groups[0]['lr']
        })
        
        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {train_loss/len(train_loader):.4f}, "
              f"Train Acc: {train_acc:.2f}%, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, "
              f"Val Acc: {val_acc:.2f}%")
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
    
    return best_val_acc

def main():
    wandb.init(project="DL_A2")
    
    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Get data loaders
    train_loader, val_loader, classes = get_dataloaders(
        data_dir="/kaggle/input/nature-12k/inaturalist_12K/train",
        batch_size=256,
        augment=True
    )
    
    # Initialize model
    model = OptimizedCNN(num_classes=len(classes))
    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer with momentum and weight decay
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-5)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    
    # Train the model
    best_val_acc = train(
        model, train_loader, val_loader, 
        optimizer, criterion, scheduler,
        device=device, epochs=20
    )
    
    wandb.summary["best_val_acc"] = best_val_acc
    wandb.finish()



In [ ]:
if __name__ == "__main__":
    main()

In [4]:
sweep_id = wandb.sweep(sweep=sweep_config, project='DL_A2')
wandb.agent(sweep_id, function=main, count=20)


Create sweep with ID: gqdzsmpm
Sweep URL: https://wandb.ai/cs24m016-indian-institute-of-technology-madras/DL_A2/sweeps/gqdzsmpm


wandb: Agent Starting Run: f0i91sgd with config:
wandb: 	activation_fn: ReLU
wandb: 	augment: False
wandb: 	batch_norm: True
wandb: 	batch_size: 256
wandb: 	conv_filters: [512, 256, 128, 64, 32]
wandb: 	dense_units: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	filter_sizes: [7, 5, 3, 3, 3]
wandb: 	lr: 0.01


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20 - Train Loss: 2.2018, Train Acc: 18.31%, Val Loss: 2.3281, Val Acc: 16.95%
Epoch 2/20 - Train Loss: 2.1250, Train Acc: 22.17%, Val Loss: 2.1231, Val Acc: 23.20%
Epoch 3/20 - Train Loss: 2.0980, Train Acc: 24.33%, Val Loss: 2.0604, Val Acc: 25.75%
Epoch 4/20 - Train Loss: 2.0716, Train Acc: 25.30%, Val Loss: 2.1120, Val Acc: 22.70%
Epoch 5/20 - Train Loss: 2.0689, Train Acc: 24.28%, Val Loss: 2.0796, Val Acc: 25.75%
Epoch 6/20 - Train Loss: 2.0404, Train Acc: 26.43%, Val Loss: 2.1111, Val Acc: 23.95%
Epoch 7/20 - Train Loss: 2.0485, Train Acc: 25.97%, Val Loss: 2.0185, Val Acc: 28.25%
Epoch 8/20 - Train Loss: 2.0112, Train Acc: 27.39%, Val Loss: 2.0595, Val Acc: 25.60%
Epoch 9/20 - Train Loss: 2.0309, Train Acc: 26.18%, Val Loss: 2.0352, Val Acc: 30.20%
Epoch 10/20 - Train Loss: 1.9918, Train Acc: 28.47%, Val Loss: 1.9891, Val Acc: 30.60%
Epoch 11/20 - Train Loss: 1.9850, Train Acc: 28.54%, Val Loss: 2.3948, Val Acc: 23.20%
Epoch 12/20 - Train Loss: 1.9786, Train Acc: 28.65%,

...

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
lr,██████████▂▂▂▂▂▂▂▂▁▁
train_accuracy,▁▂▃▃▄▄▄▅▅▅▅▆▇▇▇▇█▇██
train_loss,█▇▆▆▆▆▅▅▅▅▄▃▂▂▂▂▁▁▁▁
val_accuracy,▂▁▃▅▄▄▅▅▄▅▄▇▇▇▇█▇█▇█
val_loss,▇█▇▅▅▅▄▄▅▅▅▂▂▂▂▂▂▂▂▁
best_val_acc,39.75
epoch,20
lr,0.001
train_accuracy,37.24216
train_loss,1.77087


wandb: Agent Starting Run: 2s0ptq9w with config:
wandb: 	activation_fn: GELU
wandb: 	augment: False
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	conv_filters: [512, 256, 128, 64, 32]
wandb: 	dense_units: 256
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	filter_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.001


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20 - Train Loss: 2.2074, Train Acc: 18.06%, Val Loss: 2.2541, Val Acc: 17.55%
Epoch 2/20 - Train Loss: 2.1336, Train Acc: 22.59%, Val Loss: 2.1515, Val Acc: 22.20%
Epoch 3/20 - Train Loss: 2.0967, Train Acc: 24.35%, Val Loss: 2.0943, Val Acc: 25.00%
Epoch 4/20 - Train Loss: 2.0855, Train Acc: 25.07%, Val Loss: 2.0479, Val Acc: 27.30%
Epoch 5/20 - Train Loss: 2.0848, Train Acc: 24.29%, Val Loss: 2.1002, Val Acc: 24.60%
Epoch 6/20 - Train Loss: 2.0364, Train Acc: 26.62%, Val Loss: 1.9861, Val Acc: 29.40%
Epoch 7/20 - Train Loss: 2.0319, Train Acc: 25.62%, Val Loss: 1.9877, Val Acc: 29.35%
Epoch 8/20 - Train Loss: 2.0050, Train Acc: 27.63%, Val Loss: 1.9964, Val Acc: 28.20%
Epoch 9/20 - Train Loss: 2.0135, Train Acc: 27.05%, Val Loss: 2.0593, Val Acc: 27.25%
Epoch 10/20 - Train Loss: 1.9862, Train Acc: 28.83%, Val Loss: 2.0733, Val Acc: 25.60%
Epoch 11/20 - Train Loss: 1.9165, Train Acc: 31.39%, Val Loss: 1.8570, Val Acc: 35.00%
Epoch 12/20 - Train Loss: 1.8684, Train Acc: 33.90%,

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
lr,█████████▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▃▃▄▃▄▄▅▄▅▆▇▇▇▇▇▇███
train_loss,█▇▆▆▆▅▅▅▅▄▃▂▂▂▁▁▁▁▁▁
val_accuracy,▁▃▄▄▃▅▅▅▄▄▇▇██▇█▇█▇█
val_loss,█▇▆▅▆▄▄▄▅▅▂▂▂▂▂▁▂▁▂▁
best_val_acc,37.6
epoch,20
lr,0.01
train_accuracy,36.52957
train_loss,1.78901
